In [1]:
# Mount Google Drive to access dataset
from google.colab import drive
drive.mount('/content/drive')
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


Mounted at /content/drive


In [2]:
import tensorflow as tf
print(tf.__version__)
print("GPUs:", tf.config.list_physical_devices("GPU"))

2.18.0
GPUs: []


In [3]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [4]:
# for Vimala and Sean
%cd /content/drive/MyDrive/cs231n/student-radar/paper
# for Susan
# %cd /content/drive/MyDrive/student-radar/paper/
!pwd
!ls

/content/drive/MyDrive/cs231n/student-radar/paper
/content/drive/MyDrive/cs231n/student-radar/paper
analysis      DoOriginal.ipynb	    model-rqef7im3.h5  README.md    Test.ipynb
checkpoints   DoStudent.ipynb	    model-ryyt9way.h5  src	    wandb
data	      generated_traces.png  Plot	       summary.png
DoBoth.ipynb  model-gi5l7fcx.h5     plots	       summary.svg


In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0' # set to 1 for less verbosity (keeps warnings and errors but discards info)

In [7]:
!pip install h5py matplotlib scipy scikit-learn wandb polarTransform pyqtgraph PyQt5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 26.0 MB/s eta 0:00:00


# Generate RA Map

In [8]:
import os
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('/content/drive/MyDrive/cs231n/student-radar/paper/src/postprocessing/rota2030-bsm-rsp')

from src.Signal_Processing import Radar_Cube
from src.bsd_lib import BlindspotDetector

file_real = '/content/drive/MyDrive/cs231n/student-radar/paper/data/real/EXP_17_M.h5'
output_dir = '/content/drive/MyDrive/cs231n/student-radar/paper/plots/real_ra'
ra_output_npy = '/content/drive/MyDrive/cs231n/student-radar/paper/data/real/real_ra.npy'
os.makedirs(output_dir, exist_ok=True)

radar = Radar_Cube(file_real, dmax=30)
detector = BlindspotDetector(radar)

radar.gen_Cube(0)
radar._attrs['nLoop'] = 1

n_frames = int(radar._attrs['maxFrm'])
print(f"[INFO] Total frames in file: {n_frames}")

ra_maps = []

for frame_id in range(n_frames):
    print(f"[INFO] Processing frame {frame_id+1}/{n_frames}")

    try:
        radar.gen_Cube(frame_id)  # load one radar cube
        RP = radar.range_Profile()['Data']  # (128, 16, 1024)
        RAimg = radar.range_Azimuth()['Data']  # (Azimuth, Range)

        ra_maps.append(RAimg)

        # === Optional: save image
        plt.imsave(f"{output_dir}/frame_{frame_id:04d}.png", RAimg, cmap='viridis')
    except Exception as e:
        print(f"[WARN] Skipping frame {frame_id}: {e}")

# === Save all RA maps as .npy file
ra_maps = np.array(ra_maps)
np.save(ra_output_npy, ra_maps)
print(f"[DONE] Saved {len(ra_maps)} RA maps to: {ra_output_npy}")


[INFO] Total frames in file: 113
[INFO] Processing frame 1/113
[INFO] Processing frame 2/113
[INFO] Processing frame 3/113
[INFO] Processing frame 4/113
[INFO] Processing frame 5/113
[INFO] Processing frame 6/113
[INFO] Processing frame 7/113
[INFO] Processing frame 8/113
[INFO] Processing frame 9/113
[INFO] Processing frame 10/113
[INFO] Processing frame 11/113
[INFO] Processing frame 12/113
[INFO] Processing frame 13/113
[INFO] Processing frame 14/113
[INFO] Processing frame 15/113
[INFO] Processing frame 16/113
[INFO] Processing frame 17/113
[INFO] Processing frame 18/113
[INFO] Processing frame 19/113
[INFO] Processing frame 20/113
[INFO] Processing frame 21/113
[INFO] Processing frame 22/113
[INFO] Processing frame 23/113
[INFO] Processing frame 24/113
[INFO] Processing frame 25/113
[INFO] Processing frame 26/113
[INFO] Processing frame 27/113
[INFO] Processing frame 28/113
[INFO] Processing frame 29/113
[INFO] Processing frame 30/113
[INFO] Processing frame 31/113
[INFO] Processi

In [9]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

sys.path.append('/content/drive/MyDrive/cs231n/student-radar/paper/src/postprocessing/rota2030-bsm-rsp')

from src.Signal_Processing import Radar_Cube
from src.bsd_lib import BlindspotDetector

def generate_ra_maps(input_data, output_dir=None, output_npy=None):
    file_real = "/content/drive/MyDrive/cs231n/student-radar/paper/data/real/EXP_17_M.h5"

    os.makedirs(os.path.dirname(output_npy), exist_ok=True)

    data = np.load(input_data)
    print("[INFO] Loaded generated data:", data.shape)

    radar = Radar_Cube(file_real, dmax=30)
    _ = BlindspotDetector(radar)

    radar.gen_Cube(0)
    radar._attrs['nLoop'] = 1

    ra_maps = []

    for i, frame in enumerate(data):
        try:
            print(f"[INFO] Processing frame {i+1}/{len(data)}")

            radar.range_Profile(frame)
            RAimg = radar.range_Azimuth()['Data']
            ra_maps.append(RAimg)
        except Exception as e:
            print(f"[WARN] Skipping frame {i}: {e}")

    # Save all RA maps to .npy
    ra_maps = np.array(ra_maps)
    np.save(output_npy, ra_maps)
    print(f"[DONE] Saved {len(ra_maps)} RA maps to: {output_npy}")


In [10]:
# Original model RA map
generate_ra_maps(
    # You should be able to find the input data path from the generated data file
    input_data="/content/drive/MyDrive/cs231n/data/generated/original/model-e6756uvi/test_6000_generations.npy",
    output_dir="/content/drive/MyDrive/cs231n/student-radar/paper/plots/original_ra",
    output_npy="/content/drive/MyDrive/cs231n/student-radar/paper/data/generated/original_ra.npy",
)


Streaming output truncated to the last 5000 lines.
[INFO] Processing frame 1002/6000
[INFO] Processing frame 1003/6000
[INFO] Processing frame 1004/6000
[INFO] Processing frame 1005/6000
[INFO] Processing frame 1006/6000
[INFO] Processing frame 1007/6000
[INFO] Processing frame 1008/6000
[INFO] Processing frame 1009/6000
[INFO] Processing frame 1010/6000
[INFO] Processing frame 1011/6000
[INFO] Processing frame 1012/6000
[INFO] Processing frame 1013/6000
[INFO] Processing frame 1014/6000
[INFO] Processing frame 1015/6000
[INFO] Processing frame 1016/6000
[INFO] Processing frame 1017/6000
[INFO] Processing frame 1018/6000
[INFO] Processing frame 1019/6000
[INFO] Processing frame 1020/6000
[INFO] Processing frame 1021/6000
[INFO] Processing frame 1022/6000
[INFO] Processing frame 1023/6000
[INFO] Processing frame 1024/6000
[INFO] Processing frame 1025/6000
[INFO] Processing frame 1026/6000
[INFO] Processing frame 1027/6000
[INFO] Processing frame 1028/6000
[INFO] Processing frame 1029/60

In [11]:
# Student model RA map
generate_ra_maps(
    # You should be able to find the input data path from the generated data file from above
    input_data="/content/drive/MyDrive/cs231n/data/generated/student/model-ig3ybnf8/test_6000_generations.npy",
    output_dir="/content/drive/MyDrive/cs231n/student-radar/paper/plots/student_ra",
    output_npy="/content/drive/MyDrive/cs231n/student-radar/paper/data/generated/student_ra.npy",
)

Streaming output truncated to the last 5000 lines.
[INFO] Processing frame 1001/6000
[INFO] Processing frame 1002/6000
[INFO] Processing frame 1003/6000
[INFO] Processing frame 1004/6000
[INFO] Processing frame 1005/6000
[INFO] Processing frame 1006/6000
[INFO] Processing frame 1007/6000
[INFO] Processing frame 1008/6000
[INFO] Processing frame 1009/6000
[INFO] Processing frame 1010/6000
[INFO] Processing frame 1011/6000
[INFO] Processing frame 1012/6000
[INFO] Processing frame 1013/6000
[INFO] Processing frame 1014/6000
[INFO] Processing frame 1015/6000
[INFO] Processing frame 1016/6000
[INFO] Processing frame 1017/6000
[INFO] Processing frame 1018/6000
[INFO] Processing frame 1019/6000
[INFO] Processing frame 1020/6000
[INFO] Processing frame 1021/6000
[INFO] Processing frame 1022/6000
[INFO] Processing frame 1023/6000
[INFO] Processing frame 1024/6000
[INFO] Processing frame 1025/6000
[INFO] Processing frame 1026/6000
[INFO] Processing frame 1027/6000
[INFO] Processing frame 1028/60

KeyboardInterrupt: 

In [30]:
import numpy as np
import matplotlib.pyplot as plt

real_ra = np.load('/content/drive/MyDrive/cs231n/student-radar/paper/data/real/real_ra.npy')
original_ra = np.load('/content/drive/MyDrive/cs231n/student-radar/paper/data/generated/original_ra.npy')
student_ra = np.load('/content/drive/MyDrive/cs231n/student-radar/paper/data/generated/student_ra.npy')

frame_id = 0
real_img = real_ra[frame_id]
original_img = original_ra[frame_id]
student_img = student_ra[frame_id]

# === Plot ===
fig, axes = plt.subplots(3, 1, figsize=(10, 12), constrained_layout=True)

axes[0].imshow(real_img, aspect='auto', cmap='viridis')
axes[0].set_title("Real Data")
axes[0].set_ylabel("Range Bin")

axes[1].imshow(original_img, aspect='auto', cmap='viridis')
axes[1].set_title("Original Model Generated Data")
axes[1].set_ylabel("Range Bin")

axes[2].imshow(student_img, aspect='auto', cmap='viridis')
axes[2].set_title("Student Model Generated Data")
axes[2].set_ylabel("Range Bin")
axes[2].set_xlabel("Azimuth Bin")

plt.savefig('/content/drive/MyDrive/cs231n/student-radar/paper/plots/ra_comparison.png', dpi=500)


EOFError: No data left in file

# Signal plot


In [25]:
import numpy as np
import matplotlib.pyplot as plt

real_data_path = '/content/drive/MyDrive/cs231n/student-radar/paper/data/preprocessed/EXP_17_M_chirps.npy'
original_data_path = '/content/drive/MyDrive/cs231n/data/generated/original/model-e6756uvi/test_6000_generations.npy'
student_data_path = '/content/drive/MyDrive/cs231n/data/generated/student/model-ig3ybnf8/test_6000_generations.npy'

real_data = np.load(real_data_path)
original_data = np.load(original_data_path)
student_data = np.load(student_data_path)

frame_id = 0
real_frame = real_data[frame_id].T
original_frame = original_data[frame_id]
student_frame = student_data[frame_id]

a, b = -1, 1
train_data_path = '/content/drive/MyDrive/cs231n/student-radar/paper/data/preprocessed/EXP_17_M_chirps_scaled_train.npy'
train_data = np.load(train_data_path)

data_min = train_data.min()
data_max = train_data.max()


def denormalize(x):
    return data_min + (x - a) * (data_max - data_min) / (b - a)

original_frame_unnorm = denormalize(original_frame).T
student_frame_unnorm = denormalize(student_frame).T

In [26]:
real_frame.shape

(1024, 16)

In [27]:
original_frame_unnorm.shape

(1024, 16)

In [28]:
student_frame_unnorm.shape

(1024, 16)

In [29]:
fig, axes = plt.subplots(3, 1, figsize=(16, 18), constrained_layout=True)

for ch in range(16):
    axes[0].plot(real_frame[:, ch])
axes[0].set_title("Denormalized Real Data")
axes[0].set_ylabel("Signal")
axes[0].set_xticks([])

for ch in range(16):
    axes[1].plot(original_frame_unnorm[:, ch])
axes[1].set_title("Denormalized Original Model Generated Data")
axes[1].set_ylabel("Signal")
axes[1].set_xticks([])

for ch in range(16):
    axes[2].plot(student_frame_unnorm[:, ch])
axes[2].set_title("Denormalized Student Model Generated Data")
axes[2].set_ylabel("Signal")
axes[2].set_xlabel("Range Bin")

plt.savefig('/content/drive/MyDrive/cs231n/student-radar/paper/plots/raw_chirps_comparison.png', dpi=300)
plt.show()


Output hidden; open in https://colab.research.google.com to view.